# Steady-state Navier Stokes

In [14]:
# define geometry and generate mesh
from ngsolve import *
from ngsolve.webgui import *
from netgen.occ import *
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
geom = OCCGeometry(shape, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=0.3))

In [20]:
mesh = Mesh (geom.GenerateMesh(maxh=0.05)); nu = Parameter(1)
V = VectorH1(mesh,order=2,dirichlet="bottom|right|top|left")
Q = H1(mesh,order=1);
N = NumberSpace(mesh);
X = V*Q*N
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
a += (nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)
      -div(u)*q-div(v)*p-lam*q-mu*p)*dx

atri = BilinearForm(Trialspace=V*V, testspace=V)

In [29]:
help(a)

Help on BilinearForm in module ngsolve.comp object:

class BilinearForm(NGS_Object)
 |  Used to store the left hand side of a PDE. integrators (ngsolve.BFI)
 |  to it to implement your PDE. If the left hand side is linear
 |  you can use BilinearForm.Assemble to assemble it after adding
 |  your integrators. For nonlinear usage use BilinearForm.Apply or
 |  BilinearForm.AssembleLinearization instead of Bilinearform.Assemble.
 |  
 |  Parameters:
 |  
 |  space : ngsolve.FESpace
 |    The finite element space the bilinearform is defined on. This
 |    can be a compound FESpace for a mixed formulation.
 |  
 |  
 |   Keyword arguments can be:
 |  condense: bool = False
 |    (formerly known as 'eliminate_internal')
 |    Set up BilinearForm for static condensation of internal
 |    bubbles. Static condensation has to be done by user,
 |    this enables only the use of the members harmonic_extension,
 |    harmonic_extension_trans and inner_solve. Have a look at the
 |    documentation fo

In [21]:
gfu = GridFunction(X)
gfu.components[0].Set(CF((4*x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))

In [22]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=10, callback=lambda gfu: None):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    callback(gfu)
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du
        callback(gfu)
        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break

In [23]:
def SolveAndVisualize(multidim=True):
    gfu.components[0].Set(CF((4*x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))
    if multidim:
        gfu_it = GridFunction(gfu.space,multidim=0)
        cb = lambda gfu : gfu_it.AddMultiDimComponent(gfu.vec) # store current state
        SimpleNewtonSolve(gfu, a, callback = cb)
    else:
        SimpleNewtonSolve(gfu, a)
    Draw(gfu.components[0],mesh, vectors = {"grid_size" : 25})
    print("above you see the solution after the Newton solve.")
    if multidim:
        Draw(gfu_it.components[0], mesh, vectors = {"grid_size" : 25})
        print("above you can inspect the results after each iteration of the Newton solve (use multidim-slider).")

In [24]:
SolveAndVisualize()

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  2.802547898062877
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.007951210463583325
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  6.53514499647618e-08
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  8.314829341908468e-16


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you see the solution after the Newton solve.


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you can inspect the results after each iteration of the Newton solve (use multidim-slider).


In [25]:
nu.Set(0.01)
SolveAndVisualize(multidim=False)

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  1.0268795752516626
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.06816677002147578
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  0.005114072197127057
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  3.764873044603522e-05
Iteration   4  <A u 4 , A u 4 >_{-1}^0.5 =  2.431079711873393e-09
Iteration   5  <A u 5 , A u 5 >_{-1}^0.5 =  8.642580799498299e-17


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you see the solution after the Newton solve.


In [26]:
nu.Set(0.01)
SolveAndVisualize()

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  0.2979728380844454
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.06816677002147674
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  0.005114072197127356
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  3.76487304460598e-05
Iteration   4  <A u 4 , A u 4 >_{-1}^0.5 =  2.431079709198522e-09
Iteration   5  <A u 5 , A u 5 >_{-1}^0.5 =  9.497561260504864e-17


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you see the solution after the Newton solve.


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you can inspect the results after each iteration of the Newton solve (use multidim-slider).


In [27]:
nu.Set(0.001)
SolveAndVisualize()

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  0.1281734500727043
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.01608829849944338
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  0.10332933234259505
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  0.07542576870056626
Iteration   4  <A u 4 , A u 4 >_{-1}^0.5 =  0.13950593885792073
Iteration   5  <A u 5 , A u 5 >_{-1}^0.5 =  0.3131649033278089
Iteration   6  <A u 6 , A u 6 >_{-1}^0.5 =  0.30029712713713846
Iteration   7  <A u 7 , A u 7 >_{-1}^0.5 =  1.0602097238811419
Iteration   8  <A u 8 , A u 8 >_{-1}^0.5 =  9.064920585248915
Iteration   9  <A u 9 , A u 9 >_{-1}^0.5 =  19.12244884215964


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you see the solution after the Newton solve.


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

above you can inspect the results after each iteration of the Newton solve (use multidim-slider).


In [28]:
from ngsolve.solvers import *
nu.Set(0.001)
gfu.components[0].Set(CF((4*x*(1-x),0)),definedon=mesh.Boundaries("top"))
Newton(a,gfu,maxit=20,dampfactor=0.1)
Draw(gfu.components[0],mesh, vectors = {"grid_size" : 25})

Newton iteration  0
err =  1.1345116445211663
Newton iteration  1
err =  1.0210127245858895
Newton iteration  2
err =  0.8169900301494071
Newton iteration  3
err =  0.5725434265517533
Newton iteration  4
err =  0.34475148558548424
Newton iteration  5
err =  0.1757605336678652
Newton iteration  6
err =  0.08477519634087283
Newton iteration  7
err =  0.026892595537730907
Newton iteration  8
err =  0.00966646481878095
Newton iteration  9
err =  0.0010678300617646959
Newton iteration  10
err =  2.0919968390943966e-05
Newton iteration  11
err =  1.0127179809682405e-08
Newton iteration  12
err =  1.909113711786587e-15


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Exploring basis functions

In [12]:
# define geometry and generate mesh
from ngsolve import *
from ngsolve.webgui import *
from netgen.occ import *
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
geom = OCCGeometry(shape, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=0.3))

mesh = Mesh (geom.GenerateMesh(maxh=0.4)); # mesh with 12 vertices
Nvertices = len(mesh.vertices)
# print("number of vertices:", Nvertices)

V = VectorH1(mesh,order=2,dirichlet="bottom|right|top|left")
Q = H1(mesh,order=1);



### Vertex basis function, first component

In [13]:
print("Number of dof V: ", V.ndof)
gfvertex_1 = GridFunction(V,  multidim=Nvertices)
for i in range(Nvertices):
    gfvertex_1.vecs[i].data[:] = 0
    gfvertex_1.vecs[i].data[i] = 1.0
Draw(gfvertex_1)

Number of dof V:  74


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

### Edge basis function, first component

In [17]:
Nedges = len(mesh.edges)
gfedge_1 = GridFunction(V,  multidim=Nedges)
for i in range(Nedges):
    gfedge_1.vecs[i].data[:] = 0
    gfedge_1.vecs[i].data[Nvertices+i] = 1.0
Draw(gfedge_1, autoscale=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

### Vertex basis function, second component

In [14]:
gfvertex_2 = GridFunction(V,  multidim=Nvertices)
for i in range(Nvertices):
    gfvertex_2.vecs[i].data[:] = 0
    gfvertex_2.vecs[i].data[Nvertices+Nedges+i] = 1.0
Draw(gfvertex_2)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

### Edge basis function, second component


In [15]:
Nedges = len(mesh.edges)
gfedge_2 = GridFunction(V,  multidim=Nedges)
for i in range(Nedges):
    gfedge_2.vecs[i].data[:] = 0
    gfedge_2.vecs[i].data[2*Nvertices+Nedges+i] = 1.0
Draw(gfedge_2)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Create tensor for evaluation convection term

In [40]:

import numpy as np
mesh = Mesh (geom.GenerateMesh(maxh=0.5)); nu = Parameter(1)
V = VectorH1(mesh,order=2,dirichlet="bottom|right|top|left")
Vdim = V.ndof
V2 = V*V
u, w = V2.TrialFunction() 
v = V.TestFunction()
aconv = BilinearForm(trialspace=V2, testspace=V)
aconv += (InnerProduct(grad(w)*u,v))*dx


In [41]:
gfuw = GridFunction(V2)
A = np.zeros((Vdim, Vdim, Vdim), np.float64)
r = GridFunction(V).vec.CreateVector()
for i in range(Vdim):
    gfuw.vec.data[:] = 0
    gfuw.vec.data[i] = 1.0
    for j in range(Vdim):
        gfuw.vec.data[Vdim+j] = 1.0
        aconv.Apply(gfuw.vec, r)
        A[i,j,:] = r
        gfuw.vec.data[Vdim+j] = 0.0

In [42]:
def evaluationA(A, u):
    return u.dot(u.dot(A))

In [48]:
gfutest = GridFunction(V)
u =  np.zeros(Vdim)
u[1] = 1.0
gfutest.vec.data = u

rNout = gfutest.vec.CreateVector()
aconv.Apply(gfutest.vec, rNout)

rAout = evaluationA(A,u)
print(np.linalg.norm(rAout-rNout))

0.051412344499214245


In [49]:
print(rNout)
print(rAout)

       0
 -5.36925e-233
       0
       0
 -2.68463e-233
 -2.68463e-233
       0
       0
       0
       0
 5.36925e-234
 5.36925e-234
       0
       0
       0
       0
 2.68463e-234
       0
       0
       0
       0
       0
 3.231e-07
       0
       0
 1.6155e-07
 3.231e-07
       0
       0
       0
       0
 -2.6925e-08
 -5.385e-08
       0
       0
       0
       0
 -2.6925e-08
       0
       0
       0
       0


[ 0.          0.04166667  0.          0.          0.02083333  0.02083333
  0.          0.          0.          0.         -0.00416667 -0.00416667
  0.          0.          0.          0.         -0.00208333  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
